In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.spatial import distance
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, LineString
import pandas as pd
#from src.generate_traffic_data import clean_carla, spawn_scene, run_scene
import carla
import os
os.getcwd()
from datetime import datetime
import time
import random
from tqdm import tqdm
import networkx as nx
import numpy as np
import pickle

os.chdir('../..')
os.getcwd()

from graph_creator.MapGraph import MapGraph
from graph_creator.ActorGraph import ActorGraph

from graph_creator.plot_graphs import plot_lane_map_advanced, add_actors_to_map, add_actor_edges_to_map

### Read map graph and tracks and try to create actor graph.

In [3]:
# import pickle

# with open('/home/tmuehlen/repos/graph_coverage/data/actor_graph_argoverse.pkl', 'rb') as f:
#     ag = pickle.load(f)

# with open('/home/tmuehlen/repos/graph_coverage/data/map_argoverse.pkl', 'rb') as f:
#     map = pickle.load(f)

In [4]:
files = os.listdir("/home/tmuehlen/repos/graph_coverage/carla/data")
scn_ids = [file.split("_")[1] for file in files if "tracks" in file]
scn_ids = sorted(scn_ids)

scn_ids = [scn_id for scn_id in scn_ids if scn_id > "2025-09-05"]
print(len(scn_ids))
#scn_ids

144


In [6]:
# run through all scens so far and create the graphs:
os.makedirs("/home/tmuehlen/repos/graph_coverage/actor_graphs/carla", exist_ok=True)

for scn_id in tqdm(scn_ids):
    g_map = MapGraph()
    g_map.read_graph_from_file(f'/home/tmuehlen/repos/graph_coverage/carla/data/scene_{scn_id}_map_graph.pickle')
    tracks = pd.read_parquet(f'/home/tmuehlen/repos/graph_coverage/carla/data/scene_{scn_id}_tracks.parquet')
    tracks['road_lane_id'] = tracks.road_id.astype(str) + '_' + tracks.lane_id.astype(str)
    timestamps = tracks.timestamp.unique().tolist()
    actors = tracks.actor_id.unique().tolist()
    ag = ActorGraph()
    ag_carla = ag.from_carla_scenario(tracks, 
        g_map, 
        # max_distance_lead_veh_m=20, 
        # max_distance_neighbor_forward_m=20, 
        # max_distance_neighbor_backward_m=20, 
        # max_distance_opposite_veh_m=20,
        delta_timestep_s=1.0, # hm, not yet in the carla method?
        max_distance_lead_veh_m=50,
        max_distance_opposite_forward_m=100,
        max_distance_opposite_backward_m=10,
        max_distance_neighbor_forward_m=50,
        max_distance_neighbor_backward_m=20,
        max_node_distance_leading=10,
        max_node_distance_neighbor=6,
        max_node_distance_opposite=6,
        )
    #ag_carla.actor_graphs.keys()
    # clean up the graphs and components, i.e. remove components with only one node:
    keys = list(ag_carla.actor_graphs.keys())
    for  key in keys:
        ag_carla.actor_components[key] = [ag_carla.actor_components[key][i] for i in range(len(ag_carla.actor_components[key])) if ag_carla.actor_components[key][i].size() > 1]
    #pd.DataFrame(ag_carla.actor_graphs[5.5085996149573475].nodes(data=True)).iloc[0,1]
    ag_timestamps = list(ag_carla.actor_graphs.keys())
    timestamp_idx = 0
    component_idx = 0
    for timestamp_idx in range(len(ag_timestamps)):
        for component_idx in range(len(ag_carla.actor_components[ag_timestamps[timestamp_idx]])):
            save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/carla/graph_{scn_id}_{timestamp_idx}_{component_idx}.pkl'
            with open(save_path, "wb") as file:
                pickle.dump(ag_carla.actor_components[ag_timestamps[timestamp_idx]][component_idx], file)



100%|██████████| 144/144 [08:00<00:00,  3.34s/it]


In [ ]:
# scn_id = "2025-08-22 22:00:08.700981"
# scn_id = "2025-08-23 23:19:38.811319"
#scn_id = "2025-09-05 21:34:48.103135"
scn_id = scn_ids[34]
print(scn_id)
g_map = MapGraph()
g_map.read_graph_from_file(f'/home/tmuehlen/repos/graph_coverage/carla/data/scene_{scn_id}_map_graph.pickle')
#g_map.visualize_graph()
fig, ax = plot_lane_map_advanced(g_map.graph, color_by_length=True, cmap='plasma', show_labels=False)
plt.show()


In [ ]:
g_map.graph.nodes(data=True)
pd.DataFrame(g_map.graph.nodes(data=True))
g_map.graph.nodes(data=True)
pd.DataFrame(g_map.graph.edges(data=True)).head(2)


### load tracks data and do the actual actor graph creation

In [ ]:
tracks = pd.read_parquet(f'/home/tmuehlen/repos/graph_coverage/carla/data/scene_{scn_id}_tracks.parquet')
tracks['road_lane_id'] = tracks.road_id.astype(str) + '_' + tracks.lane_id.astype(str)
print(tracks.shape)

timestamps = tracks.timestamp.unique().tolist()
actors = tracks.actor_id.unique().tolist()
tracks.head(2).T

In [ ]:
ag = ActorGraph()
ag_carla = ag.from_carla_scenario(tracks, 
    g_map, 
    max_distance_lead_veh_m=30, 
    max_distance_neighbor_forward_m=30, 
    max_distance_neighbor_backward_m=30, 
    # max_distance_opposite_veh_m=30,
    )
ag_carla.actor_graphs.keys()


In [ ]:
# clean up the graphs and components, i.e. remove components with only one node:
keys = list(ag_carla.actor_graphs.keys())
for  key in keys:
    ag_carla.actor_components[key] = [ag_carla.actor_components[key][i] for i in range(len(ag_carla.actor_components[key])) if ag_carla.actor_components[key][i].size() > 1]

#pd.DataFrame(ag_carla.actor_graphs[5.5085996149573475].nodes(data=True)).iloc[0,1]
ag_timestamps = list(ag_carla.actor_graphs.keys())
ag_carla.actor_graphs[ag_timestamps[0]].nodes(data=True)
ag_carla.actor_graphs[ag_timestamps[0]].edges(data=True)

In [ ]:
# plot some results

# t_idx = ag_timestamps[0]
# print(t_idx)
# ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = 0, use_map_pos=True, node_size = 100, graph_or_component='graph')
# for i in range(len(ag_carla.actor_components[t_idx])):
#     if ag_carla.actor_components[t_idx][i].size() > 1:
#         ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = i, use_map_pos=True, node_size = 100, graph_or_component='component', scale_plot=False)

# type(ag_carla.actor_components[ag_timestamps[0]][0])


# timestamp_idx = 0
# component_idx = 0

# for timestamp_idx in range(len(ag_timestamps)):
#     for component_idx in range(len(ag_carla.actor_components[ag_timestamps[timestamp_idx]])):
#         save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/carla/graph_{scn_id}_{timestamp_idx}_{component_idx}.pkl'
#         with open(save_path, "wb") as file:
#             pickle.dump(ag_carla.actor_components[ag_timestamps[timestamp_idx]][component_idx], file)



In [ ]:
#  Step by step plotting
fig, ax = plot_lane_map_advanced(g_map.graph, color_by_length=True, cmap='plasma', show_labels=False)
fig, ax = add_actors_to_map(fig, ax, ag_carla.actor_graphs[ag_timestamps[0]], actor_size=20, show_actor_labels=False)
fig, ax = add_actor_edges_to_map(fig, ax, ag_carla.actor_graphs[ag_timestamps[0]])
plt.show()


### Create the timestepped graph:

In [ ]:
from graph_creator.ActorTimeGraph import ActorTimeGraph

ag_carla_time = ActorTimeGraph(ag_carla)
ag_carla_time.create_actor_time_graphs()

# clean up the graphs and components, i.e. remove components with only one node:
keys = list(ag_carla_time.actor_time_graphs.keys())
for  key in keys:
    ag_carla_time.actor_time_components[key] = [ag_carla_time.actor_time_components[key][i] for i in range(len(ag_carla_time.actor_time_components[key])) if ag_carla_time.actor_time_components[key][i].size() > 1]

ag_carla_time.actor_time_components[ag_timestamps[5]][0].nodes(data=True)

### Start comparing graphs:

In [ ]:
scn_id

In [ ]:
t_idx = ag_timestamps[11]
comp_idx = 1
G1 = ag_carla.actor_components[t_idx][comp_idx]
ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = comp_idx, use_map_pos=True, node_size = 100, graph_or_component='component')

t_idx = ag_timestamps[6]
comp_idx = 0
G2 = ag_carla.actor_components[t_idx][comp_idx]
ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = comp_idx, use_map_pos=True, node_size = 100, graph_or_component='component')

# node statistics:
print('degrees:', G1.degree())
# graph statistics:
print('density: ', nx.density(G1))
print('diameter: ', nx.diameter(G1))
#print('clustering: ', nx.average_clustering(G))


### graph comparison:

In [ ]:
print('isomorphic:', nx.is_isomorphic(G1, G2)) # actually not meaningful, not what I understood
# print('graph edit distsance:', nx.graph_edit_distance(G1, G2))
# print('graph edit distsance similarity:', nx.algorithms.similarity.graph_edit_distance(G1, G2))


In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(G1, G2)
print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G1, G2,
    node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
G1.nodes(data=True)

In [ ]:
type(G1)

### Define coverage categories in graphs:

In [ ]:
# most simple: follow lead vehicle:
from graph_creator.models import ActorType
G_scn_1 = nx.MultiDiGraph()
G_scn_1.add_node("1", actor_type = ActorType.VEHICLE)
G_scn_1.add_node("2", actor_type=ActorType.VEHICLE)
G_scn_1.add_edge("1", "2", edge_type="following_lead")

G_scn_2 = nx.MultiDiGraph()
G_scn_2.add_node("1", actor_type = ActorType.VEHICLE)
G_scn_2.add_node("2", actor_type=ActorType.VEHICLE)
G_scn_2.add_node("3", actor_type=ActorType.VEHICLE)
G_scn_2.add_edge("1", "2", edge_type="following_lead")
G_scn_2.add_edge("2", "3", edge_type="following_lead")

print(G_scn_1.nodes(data=True))
print(G_scn_1.edges(data=True))
print('')
print(G_scn_2.nodes(data=True))
print(G_scn_2.edges(data=True))
print('')
print(G1.nodes(data=True))
print(G1.edges(data=True))
print('')
print(G2.nodes(data=True))
print(G2.edges(data=True))




In [ ]:
G2_subgraph = G2.subgraph(["1777", "1767", "1768"])
print(G2_subgraph.nodes(data=True))
print(G2_subgraph.edges(data=True))


In [ ]:
g_map.graph.nodes(data=True)


In [ ]:
# Continue from here:
# Potentially add a node attribute, if the actor is on an intersection
# Define a number of subgraphs
# Loop through all scenes and count wether a scene contains a subgraph.

GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G2, G2_subgraph,
    node_match=nx.algorithms.isomorphism.categorical_node_match(['actor_type', 'lane_change'], [None, None]),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G1, G2_subgraph,
    node_match=nx.algorithms.isomorphism.categorical_node_match(['actor_type', 'lane_change'], [None, None]),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G_scn_2, G_scn_1,
    node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G1, G_scn_1,
    #node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    #edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G1, G,
    #node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.MultiDiGraphMatcher(
    G1, G,
    # node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    # edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import isomorphism

# Step 1: Create the host graph G (Directed)
G = nx.DiGraph()
G.add_weighted_edges_from([
    ('A', 'B', 1),
    ('B', 'C', 2),
    ('C', 'D', 3),
    ('A', 'D', 4),
    ('D', 'E', 2),
    ('E', 'F', 1),
])

# Step 2: Create the pattern graph H (Subgraph we're looking for)
H = nx.DiGraph()
H.add_weighted_edges_from([
    ('X', 'Y', 2),
    ('Y', 'Z', 3),
])

# Step 3: Define the edge matcher based on 'weight'
edge_matcher = isomorphism.categorical_edge_match('weight', 0)

# Step 4: Create GraphMatcher with edge attributes
GM = isomorphism.DiGraphMatcher(G, H, edge_match=edge_matcher)

# Step 5: Visualization

# Draw the host graph G
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(8, 6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=800, arrows=True)

# Label edge weights
edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Highlight matched subgraphs
if GM.subgraph_is_isomorphic():
    print("✅ H is a subgraph of G!\n📌 Matching mappings:")
    for i, mapping in enumerate(GM.subgraph_isomorphisms_iter(), 1):
        print(f"Match {i}: {mapping}")
        
        # Extract edges from the match
        matched_edges = []
        #reverse_map = {v: k for k, v in mapping.items()}
        reverse_mapping = {v: k for k, v in mapping.items()}
        for u, v in H.edges():
            if u in reverse_mapping and v in reverse_mapping:
                source = reverse_mapping[u]
                target = reverse_mapping[v]
                matched_edges.append((source, target))
        
        # Draw matched edges in red
        nx.draw_networkx_edges(G, pos, edgelist=matched_edges, edge_color='red', width=3)

else:
    print("❌ H is NOT a subgraph of G.")

plt.title("Subgraph Isomorphism with Edge Matching")
plt.show()


# Notebook junkyard